# Model

> As a developer, I like the idea of literate/explorative programming. This is why I decided to use `nbdev` to create the matlon website, using a simple graph structure.  

In [ ]:
#| default_exp model

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from IPython.display import Markdown

In [ ]:
#| export
from typing import ForwardRef
from pydantic import BaseModel
from pathlib import Path
import json
import os

In [ ]:
#| export
Project = ForwardRef('Project')

class Project(BaseModel):
    """ Model for a project and the graph of related projects """
    title: str
    categories: list[str] | None = None
    year_start: int
    year_end: int | None = None
    quote: str | None = None
    description: str | None = None
    related_project_titles: list[str] | None = None
    _related_projects: list[Project] | None = None

    def __eq__(self, other):
        if not isinstance(other, Project):
            return NotImplemented
        return self.title == other.title  # Define `title` as the unique field for equality

     # create a short_title method that returns method without spaces and without special characters, all in lowercase
    def short_title(self):
        return self.title.replace(" ", "-").replace(":", "").lower()

In [ ]:
#| export
def load_projects():
    # Load JSON data from projects.json
    # check if file nbs/projects.json exists
    path_prefix = Path("") if not os.path.exists('nbs/projects.json') else Path("nbs")

    with open(path_prefix/'projects.json', 'r', encoding='utf-8') as file:
        projects_data = json.load(file)

    # Instantiate Project objects
    projects = [Project(**data) for data in projects_data]
    projects_by_title = {project.title: project for project in projects}

    # Connect graph, load markdown files
    for project in projects:
        # if a file exists with the same name as the project and suffix .md, load it as the description
        try:
            with open(path_prefix/f'{project.short_title()}.md', 'r', encoding='utf-8') as file:
                project.description = file.read()
        except FileNotFoundError:
            try:
                with open(path_prefix/f'{project.short_title()}.mmd', 'r', encoding='utf-8') as file:
                    project.description = file.read()
            except FileNotFoundError:
                    pass
        project._related_projects = [projects_by_title[title] for title in project.related_project_titles] if project.related_project_titles else []
        # add a backlink to the related projects
        for related_project in project._related_projects:
            if not related_project._related_projects:
                related_project._related_projects = []
            if project not in related_project._related_projects:
                related_project._related_projects.append(project)

    return projects

In [ ]:
#| export
def filter_projects(projects: list[Project], category: str) -> list[Project]:
    return [project for project in projects if not category or (project.categories and category in project.categories)]

def get_categories(projects: list[Project]) -> list[str]:
    categories = set()
    for project in projects:
        if project.categories:
            categories.update(project.categories)
    return sorted(list(categories))  

In [ ]:
#| export
def projects_to_dot(projects: list[Project], category: str) -> str:
    edges = set()
    dot = 'graph G {\n'
    dot += 'rankdir=BT;\n'
    #dot += 'node [shape=box]\n'
    dot += f'category [shape="folder", color="cyan", style="filled", label="{category}"]\n'
    for project in projects:
        if project.categories and category in project.categories:
            dot += f'"{project.title}" -- category\n'
        dot += f'"{project.title}" [shape=box, URL="andri.html#{project.short_title()}"]\n'
        for related_project in project._related_projects:
            edge = "-".join(sorted([project.title, related_project.title]))
            if edge not in edges:
                edges.add(edge)
                dot += f'"{project.title}" -- "{related_project.title}" [style=dotted]\n' 
    dot += '}'
    return dot

In [ ]:
#| export
def project_to_markdown(project: Project) -> str:
    markdown = f'## {project.title}\n'
    year_range = f'{project.year_start} - {project.year_end}' if project.year_end else f'{project.year_start} - '
    markdown += f'{year_range}\n\n'
    if project.quote:
        markdown += f'\n> {project.quote}\n\n'
    if project.description:
        markdown += f'{project.description}\n\n'
    markdown += '\n### Categories\n'
    for category in project.categories:
         markdown += f'- [{category}](#category-{category})\n'
    if project._related_projects:
        markdown += '\n### Related projects\n'
        for related_project in project._related_projects:
            markdown += f'- [{related_project.title}](#{related_project.short_title()})\n'
    return markdown

run
```
nbdev_clean --fname ./nbs --clear_all
```
before commit.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()